In [1]:
import pandas as pd

In [2]:
session = spark.read.parquet("../output/ifood/ufg_dataset_all/info_session/").na.drop().dropDuplicates()
session = session.filter(session.buy == 1)
session.show(2)

+--------------------+--------------------+--------------------+-------------------+---+------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|
+--------------------+--------------------+--------------------+-------------------+---+------------+
|5ff71a38-fc11-43b...|c36939b3-5823-49b...|374d62c7-55f0-4e9...|2019-07-09 12:31:36|  1|  2019-07-09|
|0563f1a3-8099-4e1...|8a5502ce-681b-431...|374d62c7-55f0-4e9...|2019-07-23 13:57:22|  1|  2019-07-23|
+--------------------+--------------------+--------------------+-------------------+---+------------+
only showing top 2 rows



In [3]:
restaurant = spark.read.parquet("../output/ifood/ufg_dataset_all/info_restaurant/")#.na.drop().dropDuplicates()
restaurant.show(2)

+--------------------+--------------------+--------------------+----------------+-----------+---------+--------------+----------+----------+
|         merchant_id|        trading_name|         description|dish_description|price_range|avg_score|item_imagesurl|  latitude| longitude|
+--------------------+--------------------+--------------------+----------------+-----------+---------+--------------+----------+----------+
|1decb7ac-8af3-48f...|      Sushi Brothers|      Sushi Brothers| Comida Japonesa|          1|     null|          null|-23.458474|-46.514287|
|c2baed42-a1dc-44e...|Thaynam Pizzaria ...|A melhor Pizza da...|           Pizza|          3|     null|          null|-23.448811|-46.525148|
+--------------------+--------------------+--------------------+----------------+-----------+---------+--------------+----------+----------+
only showing top 2 rows



In [4]:
df = session.join(restaurant.select("merchant_id", "dish_description"), "merchant_id")
df.show()

+--------------------+--------------------+--------------------+-------------------+---+------------+----------------+
|         merchant_id|          session_id|          account_id|    click_timestamp|buy|dt_partition|dish_description|
+--------------------+--------------------+--------------------+-------------------+---+------------+----------------+
|374d62c7-55f0-4e9...|5ff71a38-fc11-43b...|c36939b3-5823-49b...|2019-07-09 12:31:36|  1|  2019-07-09| Comida Italiana|
|374d62c7-55f0-4e9...|0563f1a3-8099-4e1...|8a5502ce-681b-431...|2019-07-23 13:57:22|  1|  2019-07-23| Comida Italiana|
|374d62c7-55f0-4e9...|09ed6df7-2089-40f...|226f1138-711d-4cf...|2019-07-09 12:24:37|  1|  2019-07-09| Comida Italiana|
|374d62c7-55f0-4e9...|8a291f04-1b84-44c...|ed5c64ad-b483-428...|2019-07-18 19:29:27|  1|  2019-07-18| Comida Italiana|
|73ce4c6d-8643-4bc...|a999ecc1-e41a-49a...|bb824e1a-58e0-46b...|2019-07-17 15:58:30|  1|  2019-07-17|         Lanches|
|73ce4c6d-8643-4bc...|e10cb3cf-912d-4fa...|89df4

In [5]:
df = df.toPandas()

In [6]:
df.head(2)

,merchant_id,session_id,account_id,click_timestamp,buy,dt_partition,dish_description
0,374d62c7-55f0-4e94-9291-11ab53d7ca91,5ff71a38-fc11-43be-b314-ab2321eed14c,c36939b3-5823-49ba-b6fc-ac21103422b8,2019-07-09 12:31:36,1,2019-07-09,Comida Italiana
1,374d62c7-55f0-4e94-9291-11ab53d7ca91,0563f1a3-8099-4e12-8e11-2e08c76ee0be,8a5502ce-681b-4310-ad0e-e4d8dcb38aeb,2019-07-23 13:57:22,1,2019-07-23,Comida Italiana


In [7]:
import gc
gc.collect()

46

In [8]:
#df.groupby("dish_description").agg({'merchant_id': ''})

# interactions
# session_id_uniq
# merchant_uniq
# account_id_uniq
# days_uniq

In [48]:
df_group = df.groupby("dish_description").agg({'buy': 'count', 
                                                'session_id': 'nunique', 
                                                'merchant_id': 'nunique', 
                                                'account_id': 'nunique'}).sort_values('buy', ascending = False)
df_group

,buy,session_id,merchant_id,account_id
dish_description,,,,
Lanches,212614,65789,282,38886
Comida Brasileira,150062,57825,267,29382
Pizza,122567,41852,247,26898
Comida Japonesa,61997,18286,45,12259
Comida Chinesa,39076,12638,19,9255
Marmita,23914,9509,51,6712
Açaí,23071,7455,28,4985
Carnes,22752,8667,27,4903
Salgados,20999,6946,29,5398


In [49]:
df_group.merchant_id.sum()

1224

In [50]:
data = []

for d in list(df.dish_description.unique()):

    dff = df[df.dish_description == d]
    dff = dff[['merchant_id', 'buy']].groupby('merchant_id').count().sort_values('buy', ascending = False).head(1)#.groupby('merchant_id').first()
    dff = dff.rename(columns={'buy': 'percent_merchant_most_buy'})
    dff['dish_description'] = d
    dff = dff.set_index('dish_description')
    data.append(dff)
    
dff_m = pd.concat(data)
dff_m.head()

,percent_merchant_most_buy
dish_description,
Comida Italiana,5539
Lanches,19933
Comida Japonesa,17449
Pizza,5193
Salgados,6675


In [51]:
df_group = df_group.join(dff_m)
df_group.head()

,buy,session_id,merchant_id,account_id,percent_merchant_most_buy
dish_description,,,,,
Lanches,212614,65789,282,38886,19933
Comida Brasileira,150062,57825,267,29382,12445
Pizza,122567,41852,247,26898,5193
Comida Japonesa,61997,18286,45,12259,17449
Comida Chinesa,39076,12638,19,9255,6485


In [52]:
df_group['percent_merchant_most_buy'] = df_group['percent_merchant_most_buy']/df_group['buy']
df_group['merchant_per_account'] = df_group['merchant_id']/df_group['account_id']
df_group['account_id_per_account'] = df_group['account_id']/df_group['merchant_id']
df_group['buy_per_merchant'] = df_group['buy']/df_group['merchant_id']

df_group

,buy,session_id,merchant_id,account_id,percent_merchant_most_buy,merchant_per_account,account_id_per_account,buy_per_merchant
dish_description,,,,,,,,
Lanches,212614,65789,282,38886,0.093752,0.007252,137.893617,753.950355
Comida Brasileira,150062,57825,267,29382,0.082932,0.009087,110.044944,562.029963
Pizza,122567,41852,247,26898,0.042369,0.009183,108.898785,496.222672
Comida Japonesa,61997,18286,45,12259,0.281449,0.003671,272.422222,1377.711111
Comida Chinesa,39076,12638,19,9255,0.165959,0.002053,487.105263,2056.631579
Marmita,23914,9509,51,6712,0.371331,0.007598,131.607843,468.901961
Açaí,23071,7455,28,4985,0.191539,0.005617,178.035714,823.964286
Carnes,22752,8667,27,4903,0.531382,0.005507,181.592593,842.666667
Salgados,20999,6946,29,5398,0.317872,0.005372,186.137931,724.103448


In [53]:
df_group.to_csv('dish_description_stats.csv')